In [ ]:
import os
import numpy as np
import cv2

In [ ]:
"""
Based on the dataset_explorer.ipnyb i found the image size and the min and max values of the bounding boxed of the poles.
I can now cropp this an save some time when training
"""

def crop_image_and_label(img, label_file):
    min_x, max_x, min_y, max_y = 0, 1913, 543, 1139

    original_width, original_height = 1920, 1208

    cropped_width = max_x - min_x
    cropped_height = max_y - min_y

    cropped = img[min_y:max_y, min_x:max_x]

    new_label = []
    poles = get_poles(label_file)
    for x_norm, y_norm, w_norm, h_norm in poles:
        x_pix = x_norm * original_width
        y_pix = y_norm * original_height
        w_pix = w_norm * original_width
        h_pix = h_norm * original_height

        x_pix_cropped = x_pix - min_x
        y_pix_cropped = y_pix - min_y


        x_norm_cropped = x_pix_cropped / cropped_width
        y_norm_cropped = y_pix_cropped / cropped_height
        w_norm_cropped = w_pix / cropped_width
        h_norm_cropped = h_pix / cropped_height

        new_label.append(f"0 {x_norm_cropped:.6f} {y_norm_cropped:.6f} {w_norm_cropped:.6f} {h_norm_cropped:.6f}\n")

    return cropped, new_label


In [ ]:
"""
crop for image only
"""

def crop_image_only(img):
    min_x, max_x, min_y, max_y = 0, 1913, 543, 1139
    cropped = img[min_y:max_y, min_x:max_x]
    return cropped


In [ ]:
"""
zscore normalization, and also scale it between 0 and 255
"""

def z_score_norm(img):
    mean = np.mean(img)
    std = np.std(img)
    zscore_img = (img - mean) / std

    # don't know if this is needed, but i think it is as the images looked very bad when not applied
    zscore_rescaled = ((zscore_img - zscore_img.min()) / (zscore_img.max() - zscore_img.min()) * 255).astype(np.uint8)
    return zscore_rescaled

In [ ]:
"""
methods to get the x,y,w,h and class of each pole in each file.
"""
def get_poles(pole_files):
    poles = []
    with open(pole_files, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, x, y, w, h = map(float, parts)
                poles.append((x, y, w, h))
    return poles

In [ ]:
"""
code from epvs_segemntation from another directory in my github
that code is realted to my thesis, if needed i can me taht one accessible as well
"""

base_data_path = "../dataset/roadpoles_v1/valid"

imagesTr = os.path.join(base_data_path, "images")
labelsTr = os.path.join(base_data_path, "labels")

train_images = sorted([os.path.join(imagesTr, f) for f in os.listdir(imagesTr) if f.endswith(".PNG")])
train_labels = sorted([os.path.join(labelsTr, f) for f in os.listdir(labelsTr) if f.endswith(".txt")])

train_data = [{"image": img, "label": lbl} for img, lbl in zip(train_images, train_labels)]

output_base = "../dataset/roadpoles_v1/valid_preprocessed"
images_out = os.path.join(output_base, "images")
labels_out = os.path.join(output_base, "labels")

os.makedirs(images_out, exist_ok=True)
os.makedirs(labels_out, exist_ok=True)

preprocessed_data = []

for td in train_data:
    img = cv2.imread(td["image"])

    label = td["label"]

    cropped, new_label = crop_image_and_label(img, label)
    preprocessed_img = z_score_norm(cropped)

    img_name = os.path.basename(td["image"])
    img_save_path = os.path.join(images_out, img_name)
    cv2.imwrite(img_save_path, preprocessed_img)

    label_name = os.path.basename(td["label"])
    label_save_path = os.path.join(labels_out, label_name)
    with open(label_save_path, "w") as f:
        f.writelines(new_label)

    preprocessed_data.append({"image": img_save_path, "label": label_save_path})
